In [119]:
import re
import time
import random
import warnings
import pandas as pd
from typing import List
from lxml import etree as et
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import requests

warnings.filterwarnings('ignore')
driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://www.ulta.com/shop/fragrance/womens-fragrance/perfume?N=26wq"
driver.get(url)

def get_page_urls(url):
    page_urls = [url]
    while url:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        next_page = soup.find('li', class_='next-prev floatl-span').find('a', class_='next')
        if next_page:
            url = "https://www.ulta.com" + next_page['href']
            page_urls.append(url)
        else:
            url = None
    driver.quit()
    return set(page_urls)
    

page_urls = get_page_urls(url)
page_urls


{'https://www.ulta.com/shop/fragrance/womens-fragrance/perfume?N=26wq',
 'https://www.ulta.com/shop/fragrance/womens-fragrance/perfume?N=26wq&No=192&Nrpp=96',
 'https://www.ulta.com/shop/fragrance/womens-fragrance/perfume?N=26wq&No=288&Nrpp=96',
 'https://www.ulta.com/shop/fragrance/womens-fragrance/perfume?N=26wq&No=96&Nrpp=96'}

In [120]:
def get_product_links(page_urls: List[str]) -> List[str]:
    product_links = []
    for url in page_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        links = ["https://www.ulta.com" + row.a['href'] for row in soup.find_all('p', class_='prod-desc')]
        product_links.extend(links)
    return product_links

product_links = get_product_links(page_urls)

In [121]:
product_links

['https://www.ulta.com/p/tiffany-love-eau-de-parfum-her-pimprod2021295',
 'https://www.ulta.com/p/happy-heart-perfume-spray-VP11392',
 'https://www.ulta.com/p/flowerbomb-ruby-orchid-eau-de-parfum-pimprod2031488',
 'https://www.ulta.com/p/brit-her-eau-de-parfum-VP10147',
 'https://www.ulta.com/p/idole-aura-eau-de-parfum-pimprod2025951',
 'https://www.ulta.com/p/beautiful-magnolia-eau-de-parfum-pimprod2022124',
 'https://www.ulta.com/p/fever-eau-de-parfum-pimprod2003449',
 'https://www.ulta.com/p/miss-dior-absolutely-blooming-eau-de-parfum-xlsImpprod14631121',
 'https://www.ulta.com/p/yellow-diamond-eau-de-toilette-xlsImpprod4230037',
 'https://www.ulta.com/p/libre-eau-de-toilette-pimprod2023720',
 'https://www.ulta.com/p/libre-le-parfum-pimprod2034820',
 'https://www.ulta.com/p/flowerbomb-nectar-eau-de-parfum-pimprod2031924',
 'https://www.ulta.com/p/angel-eau-de-parfum-xlsImpprod3920101',
 'https://www.ulta.com/p/la-belle-eau-de-parfum-pimprod2034073',
 'https://www.ulta.com/p/cashmere

In [181]:
data = {
    'Product_url': [], 

}
# Creating a dataframe with those columns


df = pd.DataFrame(data)

In [182]:
for i in product_links:
    df = df.append({'Product_url': i}, ignore_index=True)

In [183]:
brands = []
product_names = []

for i in range(len(product_links)):
    product = requests.get(product_links[i])
    soup = BeautifulSoup(product.text, 'lxml')
    brand = soup.find("span", attrs={"class": "Text-ds Text-ds--body-1 Text-ds--left"}).find("a").string
    product_name = soup.find("span", attrs={"class": "Text-ds Text-ds--title-5 Text-ds--left"}).string
    brands.append(brand)
    product_names.append(product_name)
    

df['Brand'] = brands
df['Product Names'] = product_names